## Competição da base de dados do Titanic na plataforma do Kaggle.

Importação das bibliotecas, montagem do Drive e importação da base

In [0]:
#IMPORT
import pandas as pd
import numpy as np

In [2]:
# MONTANDO O DRIVE
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# BASES
train = pd.read_csv("https://raw.githubusercontent.com/RafaelLopes01/titanic-kaggle/master/data/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/RafaelLopes01/titanic-kaggle/master/data/test.csv")

## Tratamento dos Dados

In [5]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [0]:
# ADICIONANDO NOVAS VARIAVEIS NA BASE DE TREINO
train['Embarked_S'] = (train['Embarked'] == 'S').astype(int)
train['Embarked_C'] = (train['Embarked'] == 'C').astype(int)
train['Embarked_Q'] = (train['Embarked'] == 'Q').astype(int)
# ADICIONANDO NOVAS VARIAVEIS NA BASE DE TESTE
test['Embarked_S'] = (test['Embarked'] == 'S').astype(int)
test['Embarked_C'] = (test['Embarked'] == 'C').astype(int)
test['Embarked_Q'] = (test['Embarked'] == 'Q').astype(int)

In [0]:
train['Cabin_nula'] = train['Cabin'].isnull().astype(int)
test['Cabin_nula'] = test['Cabin'].isnull().astype(int)

In [0]:
# SELECIONANDO AS VARIAVEIS DA BASE DE TREINO.
variaveis_treino = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin_nula','Embarked_S','Embarked_C','Embarked_Q','Survived']

# SELECIONANDO AS VARIAVEIS DA BASE DE TESTE
variaveis_test = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin_nula','Embarked_S','Embarked_C','Embarked_Q']

In [9]:
# ATRIBUINDO AS VARIAVEIS PARA OS DATASETS.
train = train[variaveis_treino]
dados_test = test[variaveis_test]
dados_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_nula,Embarked_S,Embarked_C,Embarked_Q
0,3,male,34.5,0,0,7.8292,1,0,0,1
1,3,female,47.0,1,0,7.0000,1,1,0,0
2,2,male,62.0,0,0,9.6875,1,0,0,1
3,3,male,27.0,0,0,8.6625,1,1,0,0
4,3,female,22.0,1,1,12.2875,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,1,1,0,0
414,1,female,39.0,0,0,108.9000,0,0,1,0
415,3,male,38.5,0,0,7.2500,1,1,0,0
416,3,male,NaN,0,0,8.0500,1,1,0,0


In [0]:
train

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_nula,Embarked_S,Embarked_C,Embarked_Q,Survived
0,3,male,22.0,1,0,7.2500,1,1,0,0,0
1,1,female,38.0,1,0,71.2833,0,0,1,0,1
2,3,female,26.0,0,0,7.9250,1,1,0,0,1
3,1,female,35.0,1,0,53.1000,0,1,0,0,1
4,3,male,35.0,0,0,8.0500,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,1,1,0,0,0
887,1,female,19.0,0,0,30.0000,0,1,0,0,1
888,3,female,NaN,1,2,23.4500,1,1,0,0,0
889,1,male,26.0,0,0,30.0000,0,0,1,0,1


In [0]:
# DIVISÃO DO DATASET ENTRE PREVISORES E CLASSE.
previsores = train.iloc[: , 0:10].values
classe = train.iloc[:, 10].values
dados_test = dados_test.iloc[:, 0:10].values

In [0]:
previsores

array([[3, 'male', 22.0, ..., 1, 0, 0],
       [1, 'female', 38.0, ..., 0, 1, 0],
       [3, 'female', 26.0, ..., 1, 0, 0],
       ...,
       [3, 'female', nan, ..., 1, 0, 0],
       [1, 'male', 26.0, ..., 0, 1, 0],
       [3, 'male', 32.0, ..., 0, 0, 1]], dtype=object)

In [0]:
dados_test

array([[3, 'male', 34.5, ..., 0, 0, 1],
       [3, 'female', 47.0, ..., 1, 0, 0],
       [2, 'male', 62.0, ..., 0, 0, 1],
       ...,
       [3, 'male', 38.5, ..., 1, 0, 0],
       [3, 'male', nan, ..., 1, 0, 0],
       [3, 'male', nan, ..., 0, 1, 0]], dtype=object)

In [0]:
# TRANSFORMANDO OS ATRIBUTOS
from sklearn.preprocessing import LabelEncoder
labelencoder_previsores = LabelEncoder()
labelencoder_test = LabelEncoder()

In [0]:
previsores[:,1] = labelencoder_previsores.fit_transform(previsores[:,1])

In [0]:
dados_test[:,1] = labelencoder_test.fit_transform(dados_test[:,1])

In [0]:
dados_test

array([[3, 1, 34.5, ..., 0, 0, 1],
       [3, 0, 47.0, ..., 1, 0, 0],
       [2, 1, 62.0, ..., 0, 0, 1],
       ...,
       [3, 1, 38.5, ..., 1, 0, 0],
       [3, 1, nan, ..., 1, 0, 0],
       [3, 1, nan, ..., 0, 1, 0]], dtype=object)

In [0]:
# TRATAMENTO DOS VALORES FALTANTES NA BASE DE TREINO.
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values= np.nan, strategy= 'mean')
imputer = imputer.fit(previsores)
previsores = imputer.transform(previsores)

In [0]:
# TRATAMENTO DOS VALORES FALTANTES NA BASE DE TESTE
imputer_dados_test = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer_dados_test = imputer_dados_test.fit(dados_test)
dados_test = imputer_dados_test.transform(dados_test)

In [0]:
previsores

array([[ 3.        ,  1.        , 22.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        , 38.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 3.        ,  0.        , 26.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 3.        ,  0.        , 29.69911765, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        , 26.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 3.        ,  1.        , 32.        , ...,  0.        ,
         0.        ,  1.        ]])

In [0]:
# ESCALONAMENTO DOS PREVISORES
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)
previsores

array([[ 0.82737724,  0.73769513, -0.5924806 , ...,  0.61930636,
        -0.48204268, -0.30756234],
       [-1.56610693, -1.35557354,  0.63878901, ..., -1.61470971,
         2.0745051 , -0.30756234],
       [ 0.82737724, -1.35557354, -0.2846632 , ...,  0.61930636,
        -0.48204268, -0.30756234],
       ...,
       [ 0.82737724, -1.35557354,  0.        , ...,  0.61930636,
        -0.48204268, -0.30756234],
       [-1.56610693,  0.73769513, -0.2846632 , ..., -1.61470971,
         2.0745051 , -0.30756234],
       [ 0.82737724,  0.73769513,  0.17706291, ..., -1.61470971,
        -0.48204268,  3.25137334]])

In [0]:
# ESCALONAMENTO BASE DE TESTE
dados_test = scaler.fit_transform(dados_test)
dados_test

array([[ 0.87348191,  0.75592895,  0.3349926 , ..., -1.35067551,
        -0.56814154,  2.84375747],
       [ 0.87348191, -1.32287566,  1.32553003, ...,  0.74037028,
        -0.56814154, -0.35164743],
       [-0.31581919,  0.75592895,  2.51417495, ..., -1.35067551,
        -0.56814154,  2.84375747],
       ...,
       [ 0.87348191,  0.75592895,  0.65196458, ...,  0.74037028,
        -0.56814154, -0.35164743],
       [ 0.87348191,  0.75592895,  0.        , ...,  0.74037028,
        -0.56814154, -0.35164743],
       [ 0.87348191,  0.75592895,  0.        , ..., -1.35067551,
         1.76012477, -0.35164743]])

## Criação do Modelo

Validação Cruzada

In [0]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.30, random_state=0) 

Modelos

In [0]:
# Randon Forest
from sklearn.ensemble import RandomForestClassifier
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

In [0]:
# Rede Neural
from sklearn.neural_network import MLPClassifier
modelo = MLPClassifier(verbose = True,
                              max_iter = 100,
                              tol = 0.000010,
                              solver = 'adam',
                              hidden_layer_sizes = (100),
                              activation = 'relu')


In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
modelo = DecisionTreeClassifier(criterion='entropy', random_state=0)

In [0]:
#Regressão Logistica
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression(random_state= 1)

In [0]:
#SVM
from sklearn.svm import SVC
modelo = SVC(kernel='linear', random_state=1)

In [0]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
modelo = KNeighborsClassifier(n_neighbors= 5, metric='minkowski', p=2)

In [0]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
modelo = GaussianNB()

Treino

In [0]:
#TREINO MODELO DE TESTES
modelo.fit(previsores_treinamento, classe_treinamento)
previsoes = modelo.predict(previsores_teste)

In [0]:
#TREINO MODELO FINAL
modelo.fit(previsores, classe)
p = modelo.predict(dados_test)

Validação

In [0]:
# MATRIZ DE CONFUSÃO E ACURÁCIA DOS DADOS DE TESTES
from sklearn.metrics import confusion_matrix, accuracy_score

precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
matriz

array([[137,  31],
       [ 25,  75]])

In [0]:
precisao

0.7910447761194029

## Criar Submission


In [0]:
# CRIAR O CAMPO 'Surived' DOS DADOS DE TESTE PARA ENVIAR PARA O KAGGLE.
sub = pd.Series(p, index=test['PassengerId'], name='Survived')
sub.shape

(418,)

In [0]:
sub.to_csv("MLP_modelo.csv",header=True)

### **Resultados**
---
Randon Forest - 0.8385650224215246 | Melhorando o escalonamento - 0.8430493273542601 | Resultado Kaggle 0.76076

MLPClassifier - 0.8071748878923767 | Melhorando o escalonamento - 0.8295964125560538 | Resultado Kaggle 0.78468

DecisionTree - 0.7668161434977578

LogisticRegression - 0.7937219730941704

SVM - 0.7802690582959642

KNN - 0.7982062780269058

Naive Bayes - 0.7802690582959642
